# Chinese (Simplified) Tokenization Model - Experiments - TODO

In [4]:
import os, sys
cwd = os.getcwd()
project_path = cwd[:cwd.find('pygents')+7]
if project_path not in sys.path: sys.path.append(project_path)
os.chdir(project_path) 

from importlib import reload  # Python 3.4+

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

#force reimport
if 'pygents.util' in sys.modules:
    del sys.modules['pygents.util']
if 'pygents.text' in sys.modules:
    del sys.modules['pygents.text']
if 'pygents.plot' in sys.modules:
    del sys.modules['pygents.plot']
if 'pygents.token' in sys.modules:
    del sys.modules['pygents.token']

from pygents.util import * 
from pygents.text import * 
from pygents.plot import * 
from pygents.token import * 


In [5]:
# https://www.vengaglobal.com/blog/simplified-traditional-chinese-mandarin-cantonese/

# Target Market  Written      Spoken
# -------------------------------------
# China          Simplified   Mandarin
# Singapore      Simplified   Mandarin
# Taiwan         Traditional  Mandarin
# Hong Kong      Traditional  Cantonese

# Lexicon:
# http://www.chineselexicaldatabase.com/download.php - used below
# Sun, C. C., Hendrix, P., Ma, J.Q. & Baayen, R. H. (2018). Chinese Lexical Database (CLD): A large-scale lexical database for simplified Mandarin Chinese. Behavior Research Methods, https://doi.org/10.3758/s13428-018-1038-3.

# Corpora:
# https://www.openslr.org/38/ - test-audio corpus, not relevant
# https://github.com/CLUEbenchmark/CLUECorpus2020/ - email request sent
# https://github.com/brightmart/nlp_chinese_corpus - nearly same as above downloaded, used further

# TODO:
# https://metatext.io/datasets/nlp-chinese-corpus - paper with word segmentation


In [6]:
path = '../../nlp/corpora/Chinese/'


In [4]:
def zh_clue_json2text(path,filename):
    with open(os.path.join(path,filename+'.json')) as file:
        with open(os.path.join(path,filename+'.txt'), 'w') as fout:
            while True:
                line = file.readline()
                if not line:
                    break
                j = json.loads(line)
                #print('title',j['title'])
                #print('desc',j['desc'])
                #print('content',j['content'])
                fout.write(j['title'])
                fout.write('\n')    
                fout.write(j['desc'])
                fout.write('\n')    
                fout.write(j['content'])
                fout.write('\n')    


In [5]:
#do this once!
#zh_clue_json2text(path,'clue/new2016zh/news2016zh_valid')


In [6]:
#do this once!
#zh_clue_json2text(path,'clue/new2016zh/news2016zh_train')


## Load and explore full models

In [3]:
zh_valid_chars = FreedomTokenizer(max_n=3,mode='chars',debug=False)
#zh_valid_grams = FreedomTokenizer(max_n=3,mode='grams',debug=False)

with open(join(path, 'clue/new2016zh/news2016zh_valid.txt'),errors='ignore') as f:
    cnt = 0
    while True:
        line = f.readline()
        if not line:
            break
        cnt += 1
        if (cnt % 1000) == 0:
            print(cnt)
        zh_valid_chars.train([line])
        #zh_valid_grams.train([line])

zh_valid_chars.store('data/models/zh_valid_chars_3a')
#zh_valid_grams.store('data/models/zh_valid_grams_3a')

print(zh_valid_chars.count_params())
# 143,129,564

#print(zh_valid_grams.count_params())


NameError: name 'path' is not defined

In [8]:
del zh_valid_chars

In [ ]:
zh_train_chars = FreedomTokenizer(max_n=2,mode='chars',debug=False)

with open(join(path, 'clue/new2016zh/news2016zh_train.txt'),errors='ignore') as f:
    cnt = 0
    while True:
        line = f.readline()
        if not line:
            break
        cnt += 1
        if (cnt % 100000) == 0:
            print(cnt)
        zh_train_chars.train([line])
        #zh_valid_grams.train([line])

zh_train_chars.store('data/models/zh_train_chars_2a')

print(zh_train_chars.count_params())
# runs out of memory at 45G


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000


In [ ]:
#del zh_train_chars